<a href="https://colab.research.google.com/github/rcpbayindir/A_Guide_to_Running_Tensorflow_Models_on_Android/blob/master/Copy_of_MNIST_TransferLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Hands on #2**
# Transfer Learning Using MNIST Dataset
-------------------------------------------------

In [1]:
import keras
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.layers import Input, Flatten, Dense
from keras.models import Model, load_model

Using TensorFlow backend.


# Loading and Preprocessing MNIST dataset

In [2]:
from keras.datasets import mnist

# loading MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize the dataset
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

################ Preprocessing ##############
import cv2
import numpy as np

# Converting images to RGB
x_train = [cv2.cvtColor(cv2.resize(i, (32,32)), cv2.COLOR_GRAY2BGR) for i in x_train]
x_train = np.concatenate([arr[np.newaxis] for arr in x_train]).astype('float32')

x_test = [cv2.cvtColor(cv2.resize(i, (32,32)), cv2.COLOR_GRAY2BGR) for i in x_test]
x_test = np.concatenate([arr[np.newaxis] for arr in x_test]).astype('float32')   

# Convert labels to categorical
from keras.utils import to_categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

11493376/11490434 [==============================] - 1s 0us/step


# Transfer Learning using VGG16 trained on imagenet

In [0]:
model_vgg16_conv = VGG16(weights='imagenet', include_top=False)
model_vgg16_conv.summary()

58892288/58889256 [==============================] - 6s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
______________________________________________________________

In [0]:
# Make vgg16 model layers as non trainable
for layer in model_vgg16_conv.layers:
   layer.trainable = False
    
model_vgg16_conv.summary()    

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

**Freeze the weights of all of the layers**


In [0]:
img_shape = x_train.shape[1:]
num_classes = 10

#Create your own input format
keras_input = Input(shape=img_shape, name = 'image_input')

#Use the generated model 
output_vgg16_conv = model_vgg16_conv(keras_input)

# Add the fully-connected layers 
My_vgg16 = Flatten(name='flatten')(output_vgg16_conv)
My_vgg16 = Dense(256, activation='relu', name='fc1')(My_vgg16)
My_vgg16 = Dense(64, activation='relu', name='fc2')(My_vgg16)
My_vgg16 = Dense(num_classes, activation='softmax', name='predictions')(My_vgg16)

#Create your own model 
pretrained_model = Model(inputs=keras_input, outputs=My_vgg16)
pretrained_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
pretrained_model.fit(x_train, y_train, batch_size=32, epochs=2)

# Evaluate the model
score = pretrained_model.evaluate(x_test, y_test)
print("Accuracy on test set: ", score[1])


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/2
60000/60000 [==============================] - 42s 705us/step - loss: 0.2799 - acc: 0.9140
Epoch 2/2
10000/10000 [==============================] - 6s 589us/step
Accuracy on test set:  0.9633


**Freeze the weights of all of the layers except last 4 layers**

In [0]:
# Make vgg16 model layers as non trainable
for layer in model_vgg16_conv.layers[-4:]:
   layer.trainable = True

model_vgg16_conv.summary()

In [0]:
img_shape = x_train.shape[1:]

#Create your own input format
keras_input = Input(shape=img_shape, name = 'image_input')

#Use the generated model 
output_vgg16_conv = model_vgg16_conv(keras_input)

# Add the fully-connected layers 
My_vgg16 = Flatten(name='flatten')(output_vgg16_conv)
My_vgg16 = Dense(256, activation='relu', name='fc1')(My_vgg16)
My_vgg16 = Dense(64, activation='relu', name='fc2')(My_vgg16)
My_vgg16 = Dense(num_classes, activation='softmax', name='predictions')(My_vgg16)

#Create your own model 
pretrained_model = Model(inputs=keras_input, outputs=My_vgg16)
pretrained_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
pretrained_model.fit(x_train, y_train, batch_size=32, epochs=2)

# Evaluate the model
score = pretrained_model.evaluate(x_test, y_test)
print("Accuracy on test set: ", score[1])